# 简介
本项目是参加飞桨常规赛：中文场景文字识别的项目，项目score为85.68143。

生成的预测文件为/home/aistudio/result.txt

项目任务为识别包含中文文字的街景图片，准确识别图片中的文字

本项目源于
https://aistudio.baidu.com/aistudio/projectdetail/681670

感谢开发者为开源社区做出的贡献

# 赛题说明
**赛题背景**

中文场景文字识别技术在人们的日常生活中受到广泛关注，具有丰富的应用场景，如：拍照翻译、图像检索、场景理解等。然而，中文场景中的文字面临着包括光照变化、低分辨率、字体以及排布多样性、中文字符种类多等复杂情况。如何解决上述问题成为一项极具挑战性的任务。

本次飞桨常规赛以 中文场景文字识别 为主题，由2019第二届中国AI+创新创业全国大赛降低难度而来，提供大规模的中文场景文字识别数据，旨在为研究者提供学术交流平台，进一步推动中文场景文字识别算法与技术的突破。

**比赛任务**

要求选手必须使用飞桨对图像区域中的文字行进行预测，返回文字行的内容。

**数据集介绍**

本次竞赛数据集共包括33万张图片，其中21万张图片作为训练集，12万张作为测试集。数据集采自中国街景，并由街景图片中的文字行区域（例如店铺标牌、地标等等）截取出来而形成。所有图像都经过一些预处理，将文字区域利用仿射变化，等比映射为一张高为48像素的图片，如下图1所示：

![](https://ai-studio-static-online.cdn.bcebos.com/fb3cf59747e04f0cb9adde6a5a1945b3d9ef82f3b7c14c98bf248eb1c3886a3f)


(a) 标注：魅派集成吊顶

![](https://ai-studio-static-online.cdn.bcebos.com/57d58a35e1f34278bdb013b3f945ab69cddacf37c7fe40deba3c124fa1249753)


(b) 标注：母婴用品连锁
图1

**标注文件**

平台提供的标注文件为.txt文件格式。样例如下：



| h | w | name | value |
| -------- | -------- | -------- |-------- |
| 128 | 48 | img_1.jpg | 文本1|
| 56	| 48	| img_2.jpg|	文本2|
其中，文件中的四列分别是图片的宽、高、文件名和文字标注。

# 安装第三方库
注意：若项目重启后无法运行程序，尝试重新安装如下相应的依赖。

In [ ]:
! mkdir /home/aistudio/external-libraries
! pip install --upgrade pip -t /home/aistudio/external-libraries
! pip install tqdm paddlepaddle-gpu==1.7.1.post97 -i https://mirror.baidu.com/pypi/simple -t /home/aistudio/external-libraries
! pip install pqi -t /home/aistudio/external-libraries
! pqi use aliyun -t /home/aistudio/external-libraries
! pip install tqdm imgaug lmdb matplotlib opencv-python Pillow python-Levenshtein PyYAML trdg anyconfig -t /home/aistudio/external-libraries

# 处理数据集
* 项目重启后，必须要重新解压数据，压缩包内含训练集图片、训练集图片信息、测试集图片， 否则无法运行本程序


In [ ]:
import sys
sys.path.append('/home/aistudio/external-libraries')
import os
os.chdir('/home/aistudio/data/data10879')
! rm -rf /test_images/
os.chdir('/home/aistudio/work')
! unzip test_images.zip
! mv /home/aistudio/work/test_images/ /home/aistudio/data/data10879
! rm -rf /home/aistudio/work/__MACOSX/
! rm -rf /home/aistudio/work/test_images/
os.chdir('/home/aistudio/data/data10879')
! rm -rf /train_img/
! tar -zxf train_img.tar.gz

# 预处理

* 项目重启后，需要重新做数据预处理
* 文件 langconv(language convert)，这个文件用来把繁体字转成简体字<br>

* 函数 read_ims_list：读取train.list文件，生成图片的信息字典
* 函数 modify_ch：对标签label进行修改，进行四项操作，分别是“繁体->简体”、“大写->小写”、“删除空格”、“删除符号”。
* 函数 pipeline：调用定义的函数，对训练数据进行初步处理。

In [ ]:
from work.langconv import Converter
import codecs
import random
import sys
import os
from os.path import join as pjoin

os.chdir('/home/aistudio')
sys.path.append('/home/aistudio/work')
def read_ims_list(path_ims_list):
    """
    读取 train.list 文件
    """
    ims_info_dic = {}
    with open(path_ims_list, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(maxsplit=3)
            w, h, file, label = parts[0], parts[1], parts[2], parts[3]
            ims_info_dic[file] = {'label': label, 'w': int(w)}
    return ims_info_dic
    

def modify_ch(label):
    # 繁体 -> 简体
    label = Converter("zh-hans").convert(label)

    # 大写 -> 小写
    label = label.lower()

    # 删除空格
    label = label.replace(' ', '')

    # 删除符号
    for ch in label:
        if (not '\u4e00' <= ch <= '\u9fff') and (not ch.isalnum()):
            label = label.replace(ch, '')

    return label

def save_txt(data, file_path):
    """
    将一个list的数组写入txt文件里
    :param data:
    :param file_path:
    :return:
    """
    if not isinstance(data, list):
        data = [data]
    with open(file_path, mode='w', encoding='utf8') as f:
        f.write('\n'.join(data))

def pipeline(dataset_dir):
    path_ims        = pjoin(dataset_dir, "train_images")
    path_ims_list   = pjoin(dataset_dir, "train.list")
    path_train_list = pjoin('/home/aistudio/work', "train.txt")
    path_test_list  = pjoin('/home/aistudio/work', "test.txt")
    path_label_list = pjoin('/home/aistudio/work', "dict.txt")

    # 读取数据信息
    file_info_dic = read_ims_list(path_ims_list)

    # 创建 train.txt
    class_set = set()
    data_list = []
    for file, info in file_info_dic.items():
        label = info['label']
        label = modify_ch(label)

        # 异常: 标签为空
        if label == '':
            continue

        for e in label:
            class_set.add(e)
        data_list.append("{0}\t{1}".format(pjoin('/home/aistudio/',path_ims, file), label))
        
    # 创建 label_list.txt
    class_list = list(class_set)
    class_list.sort()
    print("class num: {0}".format(len(class_list)))
    with codecs.open(path_label_list, "w", encoding='utf-8') as label_list:
        for id, c in enumerate(class_list):
            # label_list.write("{0}\t{1}\n".format(c, id))
            label_list.write("{0}\n".format(c))

    # 随机切分
    random.shuffle(data_list)
    val_len = int(len(data_list) * 0.05)
    val_list = data_list[-val_len:]
    train_list = data_list[:-val_len]
    print('训练集数量: {}, 验证集数量: {}'.format(len(train_list),len(val_list)))
    save_txt(train_list,path_train_list)
    save_txt(val_list,path_test_list)
    
random.seed(0)
pipeline(dataset_dir="data/data10879")

### 对于PaddleOCR提供的配置文件rec_r34_vd_none_bilstm_ctc.yml我做出了如下修改

1.将epoch_num改为30

2.将train_batch_size_per_card改为256

3.将test_batch_size_per_card改为128

4.将base_lr改为0.00001

经测试这样能提高score

# 启动训练

In [ ]:
import os
os.chdir('/home/aistudio/work/PaddleOCR/')
! pwd
! export PYTHONPATH=$PYTHONPATH:.
! python tools/train.py -c configs/rec/rec_r34_vd_none_bilstm_ctc.yml

# 启动测试
***注意,没有模型文件的情况下请先进行训练再进行测试***

In [ ]:
import os
os.chdir('/home/aistudio/work/PaddleOCR/')
! pwd
! export PYTHONPATH=$PYTHONPATH:.
! python tools/infer_rec.py -c configs/rec/rec_r34_vd_none_bilstm_ctc.yml -o Global.checkpoints=output/rec_CRNN_aug_341/latest
#! python tools/export_model.py -c "configs/rec/rec_r34_vd_none_bilstm_ctc.yml" 

# **方案说明书**
# 1. 比赛介绍+赛题重点难点剖析

* 比赛介绍：本项目是参加飞桨常规赛：中文场景文字识别的项目，项目任务为识别包含中文文字的街景图片，准确识别图片中的文字。

* 赛题重点难点剖析：中文场景中的文字面临着包括光照变化、低分辨率、字体以及排布多样性、中文字符种类多等复杂情况。如何解决上述问题成为一项极具挑战性的任务。

# 2.  思路介绍+方案亮点
* 思路介绍：第一步是针对中文场景下的数据预处理（包括：把繁体字转成简体字，大写->小写，删除空格，删除符号等操作），结合相应的中文字典来提升文字识别的准确率。第二步是在飞桨框架下采用当前业界最经典的CRNN算法架构来建模与求解，以保证模型的性能。

* 方案亮点：结合中文场景下的字典资源来完成数据的预处理，可以更好的构建训练模型的语料;使用了飞桨的PaddleOCR模块,通过Fine-Tune预训练模型的方式达到快速训练和预测并且获得很好的预测效果

# 3. 具体方案分享
使用了飞桨PaddleOCR提供的rec_r34_vd_none_bilstm_ctc.yml配置文件,修改如下:
```
Global:
  algorithm: CRNN
  use_gpu: true
  epoch_num: 30
  log_smooth_window: 20
  print_batch_step: 100
  save_model_dir: output/rec_CRNN_aug_341
  save_epoch_step: 1
  eval_batch_step: 1800
  train_batch_size_per_card: 256
  test_batch_size_per_card: 128
  image_shape: [3, 32, 256]
  max_text_length: 64
  character_type: ch
  loss_type: ctc
  reader_yml: ./configs/rec/rec_icdar15_reader.yml
  pretrain_weights: /home/aistudio/work/PaddleOCR/model/latest
  checkpoints: /home/aistudio/work/PaddleOCR/output/rec_CRNN_aug_341/latest
  save_inference_dir: /home/aistudio/work/test
  character_dict_path: /home/aistudio/work/dict.txt
  infer_img: /home/aistudio/data/data10879/test_images
  
Architecture:
  function: ppocr.modeling.architectures.rec_model,RecModel

Backbone:
  function: ppocr.modeling.backbones.rec_resnet_vd,ResNet
  layers: 34
 
Head:
  function: ppocr.modeling.heads.rec_ctc_head,CTCPredict
  encoder_type: rnn
  SeqRNN:
    hidden_size: 256
    
Loss:
  function: ppocr.modeling.losses.rec_ctc_loss,CTCLoss

Optimizer:
  function: ppocr.optimizer,AdamDecay
  base_lr: 0.00001
  beta1: 0.9
  beta2: 0.999
```

# 4. 模型应用结果分析
* 不同模型结果的对比分析：

| 模型名称 | score | norm_distance |word_acc |
| -------- | -------- | -------- | -------- |
| 官方基线模型（PaddleOCR：中文场景文字识别）| 82.87     | 0.93946     | 0.82872     |
|CRNN rec_r34_vd_none_bilstm_ctc | 85.68143    | 0.95619    | 0.85681     |
* 调参优化过程分析
1. 将epoch_num改为30 
2. 将train_batch_size_per_card改为256 
3. 将test_batch_size_per_card改为128 
4. 将base_lr改为0.00001 经测试这样能提高score

# 5.  总结+改进完善方向
* 总结 
 第一次参加OCR方面的比赛,扩宽了自己的眼界,使自己对计算机视觉、深度学习模型等有了更深刻的认识,学习了很多调整参数的技巧,阅读了很多国内外的相关论文文献,本次比赛的模型可以用在中文场景文字识别的实际应用中.
* 改进完善方向
1.  CNN部分目前用的RESNET，后续可以考虑改成VGG网络；
2.  可以采取本地生成数据集的方式进行数据增强
3.  后续可以进一步优化损失函数和训练策略，以便提升模型的收敛速度。

#  7.  参考资料
1. EAST:
@inproceedings{zhou2017east,
  title={EAST: an efficient and accurate scene text detector},
  author={Zhou, Xinyu and Yao, Cong and Wen, He and Wang, Yuzhi and Zhou, Shuchang and He, Weiran and Liang, Jiajun},
  booktitle={Proceedings of the IEEE conference on Computer Vision and Pattern Recognition},
  pages={5551--5560},
  year={2017}
}

2. DB:
@article{liao2019real,
  title={Real-time Scene Text Detection with Differentiable Binarization},
  author={Liao, Minghui and Wan, Zhaoyi and Yao, Cong and Chen, Kai and Bai, Xiang},
  journal={arXiv preprint arXiv:1911.08947},
  year={2019}
}

3. DTRB:
@inproceedings{baek2019wrong,
  title={What is wrong with scene text recognition model comparisons? dataset and model analysis},
  author={Baek, Jeonghun and Kim, Geewook and Lee, Junyeop and Park, Sungrae and Han, Dongyoon and Yun, Sangdoo and Oh, Seong Joon and Lee, Hwalsuk},
  booktitle={Proceedings of the IEEE International Conference on Computer Vision},
  pages={4715--4723},
  year={2019}
}

4. SAST:
@inproceedings{wang2019single,
  title={A Single-Shot Arbitrarily-Shaped Text Detector based on Context Attended Multi-Task Learning},
  author={Wang, Pengfei and Zhang, Chengquan and Qi, Fei and Huang, Zuming and En, Mengyi and Han, Junyu and Liu, Jingtuo and Ding, Errui and Shi, Guangming},
  booktitle={Proceedings of the 27th ACM International Conference on Multimedia},
  pages={1277--1285},
  year={2019}
}

5. SRN:
@article{yu2020towards,
  title={Towards Accurate Scene Text Recognition with Semantic Reasoning Networks},
  author={Yu, Deli and Li, Xuan and Zhang, Chengquan and Han, Junyu and Liu, Jingtuo and Ding, Errui},
  journal={arXiv preprint arXiv:2003.12294},
  year={2020}
}

6. end2end-psl:
@inproceedings{sun2019chinese,
  title={Chinese Street View Text: Large-scale Chinese Text Reading with Partially Supervised Learning},
  author={Sun, Yipeng and Liu, Jiaming and Liu, Wei and Han, Junyu and Ding, Errui and Liu, Jingtuo},
  booktitle={Proceedings of the IEEE International Conference on Computer Vision},
  pages={9086--9095},
  year={2019}
}